In [ ]:
from datetime import datetime
from summer2.utils import Epoch
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"

from aust_covid.inputs import load_household_impacts_data
from summer2.functions.time import get_linear_interpolation_function

In [ ]:
# Get data, using test to symptomatic ratio
hh_impact = load_household_impacts_data()
hh_test_ratio = hh_impact["test_prop"] / hh_impact["sympt_prop"]

In [ ]:
hh_impact.plot(labels={"value": "Reporting percentage", "index": ""})

In [ ]:
# Get function that interpolates through data points (can get this from the model object)
aust_epoch = Epoch(datetime(2019, 12, 31))
ratio_interp = get_linear_interpolation_function(aust_epoch.dti_to_index(hh_test_ratio.index), hh_test_ratio.to_numpy())

In [ ]:
# Get some x-values and look a outputs
eval_start = aust_epoch.datetime_to_number(datetime(2022, 1, 1))
eval_end = aust_epoch.datetime_to_number(datetime(2022, 12, 31))
n_times = 40
eval_times = np.linspace(eval_start, eval_end, n_times)
out_array = pd.Series(
    [float(ratio_interp.evaluate(model_variables={"time": t})) for t in eval_times], 
    index=aust_epoch.index_to_dti(eval_times),
)
out_array.plot(labels={"value": "ratio", "index": ""}, title="Test to symptomatic ratio over time")

In [ ]:
def get_param_to_exp_plateau(
    input_request: float, 
    output_request: float,
) -> float:
    """
    Get the parameter needed to ensure the function:
    output = 1 - exp(-param * input)
    passes through the requested input and output.
    
    Args:
        input_request: Independent variable at known point
        output_request: Dependent variable at known point
    """
    return -np.log(1.0 - output_request) / input_request

In [ ]:
# Sample the starting CDR from a normal distribution
n_samples = 20
min_cdr = 0.06
max_cdr = 0.14
start_cdr_samples = np.random.rand(n_samples) * (max_cdr - min_cdr) + min_cdr

In [ ]:
# Get the corresponding parameter values for the exponential function
exp_param = get_param_to_exp_plateau(hh_test_ratio[0], start_cdr_samples)

# Collate starting CDRs with ratios
ratio_to_cdr_df = pd.DataFrame(columns=start_cdr_samples)
for i in range(len(start_cdr_samples)):
    cdr_values = 1.0 - np.exp(-hh_test_ratio * exp_param[i])
    ratio_to_cdr_df[start_cdr_samples[i]] = pd.Series(list(cdr_values), index=hh_test_ratio)
ratio_to_cdr_df.plot(markers=True, labels={"value": "CDR", "index": "ratio"}, title="Ratio to modelled CDR value")

In [ ]:
date_to_cdr = ratio_to_cdr_df
date_to_cdr.index = hh_impact.index
date_to_cdr.plot(markers=True, labels={"value": "CDR", "index": ""}, title="Modelled CDR over time")